# Australian Property Investment Analysis
## Comprehensive Road Accessibility & Market Intelligence Score

This analysis creates a data-driven property investment metric for Australian residential investors by combining:
- **Transport Infrastructure** (road connectivity & accessibility)
- **Market Activity** (transaction volume & property values) 
- **Development Density** (growth potential indicators)
- **Spatial Analysis** (cadastral boundaries & geographic relationships)

**Target Users**: Australian property investors seeking objective, location-based investment insights

In [3]:
# Import all required libraries
import pandas as pd
import geopandas as gpd
import numpy as np
import pyarrow.parquet as pq
from sklearn.preprocessing import MinMaxScaler
from shapely import wkt

print("📦 All libraries imported successfully!")

📦 All libraries imported successfully!


In [4]:
# Load all four datasets from the Data folder
print("=== LOADING ALL DATASETS ===")
print()

# 1. Cadastral boundaries (property parcels)
cadastre = gpd.read_file('Data/cadastre.gpkg')
print(f"✅ Cadastre parcels: {cadastre.shape[0]:,} property boundaries loaded")

# 2. Road network 
roads = gpd.read_file('Data/roads.gpkg')
print(f"✅ Road network: {roads.shape[0]:,} road segments loaded")

# 3. GNAF property addresses (using pyarrow for parquet files)
try:
    gnaf_prop = pd.read_parquet('Data/gnaf_prop.parquet')
except:
    # Alternative method if needed
    table = pq.read_table('Data/gnaf_prop.parquet')
    gnaf_prop = table.to_pandas()
print(f"✅ GNAF properties: {gnaf_prop.shape[0]:,} geocoded addresses loaded")

# 4. Property transactions
try:
    transactions = pd.read_parquet('Data/transactions.parquet')
except:
    # Alternative method if needed  
    table = pq.read_table('Data/transactions.parquet')
    transactions = table.to_pandas()
print(f"✅ Property transactions: {transactions.shape[0]:,} sales records loaded")

print()
print("🎯 DATASET SUMMARY:")
print(f"   • Property parcels: {len(cadastre):,}")
print(f"   • Road segments: {len(roads):,}")
print(f"   • Property addresses: {len(gnaf_prop):,}")
print(f"   • Transaction records: {len(transactions):,}")
print(f"   • Geographic coverage: {', '.join(cadastre['state'].unique())}")

=== LOADING ALL DATASETS ===

✅ Cadastre parcels: 1,294 property boundaries loaded
✅ Road network: 173 road segments loaded
✅ GNAF properties: 70,591 geocoded addresses loaded
✅ Property transactions: 5,576 sales records loaded

🎯 DATASET SUMMARY:
   • Property parcels: 1,294
   • Road segments: 173
   • Property addresses: 70,591
   • Transaction records: 5,576
   • Geographic coverage: NSW


In [5]:
# Data preparation and coordinate system setup
print("=== DATA PREPARATION ===")
print()

# Convert date columns for transactions
if 'date_sold' in transactions.columns:
    transactions['date_sold'] = pd.to_datetime(transactions['date_sold'], errors='coerce')

# Check coordinate reference systems
print("Coordinate Reference Systems:")
print(f"Cadastre CRS: {cadastre.crs}")
print(f"Roads CRS: {roads.crs}")

# Convert to Australian Albers projection for accurate spatial calculations
cadastre_projected = cadastre.to_crs('EPSG:3577')  # Australian Albers
roads_projected = roads.to_crs('EPSG:3577')

# Calculate parcel areas
cadastre_projected['area_sqm'] = cadastre_projected.geometry.area
cadastre_projected['area_hectares'] = cadastre_projected['area_sqm'] / 10000

# Create GNAF spatial dataset
gnaf_gdf = gpd.GeoDataFrame(
    gnaf_prop,
    geometry=gpd.points_from_xy(gnaf_prop.longitude, gnaf_prop.latitude),
    crs='EPSG:4326'
)
gnaf_projected = gnaf_gdf.to_crs('EPSG:3577')

print()
print("📊 PROPERTY SIZE DISTRIBUTION:")
print(f"   • Mean parcel size: {cadastre_projected['area_hectares'].mean():.2f} hectares")
print(f"   • Median parcel size: {cadastre_projected['area_hectares'].median():.2f} hectares")
print(f"   • Size range: {cadastre_projected['area_hectares'].min():.2f} - {cadastre_projected['area_hectares'].max():.2f} hectares")
print()
print("🏠 MARKET DATA OVERVIEW:")
print(f"   • Transaction date range: {transactions['date_sold'].min()} to {transactions['date_sold'].max()}")
print(f"   • Average property price: ${transactions['price'].mean():,.0f}")
print(f"   • Median property price: ${transactions['price'].median():,.0f}")

=== DATA PREPARATION ===

Coordinate Reference Systems:
Cadastre CRS: EPSG:4326
Roads CRS: EPSG:4326

📊 PROPERTY SIZE DISTRIBUTION:
   • Mean parcel size: 0.15 hectares
   • Median parcel size: 0.07 hectares
   • Size range: 0.00 - 35.30 hectares

🏠 MARKET DATA OVERVIEW:
   • Transaction date range: 2002-12-19 00:00:00 to 2025-07-03 00:00:00
   • Average property price: $1,580,499
   • Median property price: $1,180,000

📊 PROPERTY SIZE DISTRIBUTION:
   • Mean parcel size: 0.15 hectares
   • Median parcel size: 0.07 hectares
   • Size range: 0.00 - 35.30 hectares

🏠 MARKET DATA OVERVIEW:
   • Transaction date range: 2002-12-19 00:00:00 to 2025-07-03 00:00:00
   • Average property price: $1,580,499
   • Median property price: $1,180,000


In [6]:
# Calculate comprehensive spatial metrics for each property parcel
print("=== CALCULATING SPATIAL ACCESSIBILITY METRICS ===")
print()

# Initialize the main analysis dataframe
accessibility_metrics = cadastre_projected.copy()

# Define buffer distances for road accessibility analysis
buffer_distances = [100, 250, 500]  # meters

print("Calculating road accessibility for multiple buffer zones...")

for buffer_dist in buffer_distances:
    print(f"   • Processing {buffer_dist}m buffer zone...")
    
    # Create buffer around each parcel
    parcel_buffers = cadastre_projected.geometry.buffer(buffer_dist)
    
    # Initialize lists for this buffer distance
    roads_in_buffer = []
    total_road_length = []
    major_road_count = []
    
    for i, buffer_geom in enumerate(parcel_buffers):
        # Find roads intersecting with this buffer
        intersecting_roads = roads_projected[roads_projected.geometry.intersects(buffer_geom)]
        
        roads_in_buffer.append(len(intersecting_roads))
        
        # Calculate road metrics within buffer
        if len(intersecting_roads) > 0:
            clipped_roads = intersecting_roads.geometry.intersection(buffer_geom)
            total_length = clipped_roads.length.sum()
            total_road_length.append(total_length)
            
            # Count major roads (primary, secondary, trunk highways)
            major_roads = intersecting_roads[intersecting_roads['fclass'].isin(['primary', 'secondary', 'trunk'])]
            major_road_count.append(len(major_roads))
        else:
            total_road_length.append(0)
            major_road_count.append(0)
    
    # Add metrics to dataframe
    accessibility_metrics[f'roads_count_{buffer_dist}m'] = roads_in_buffer
    accessibility_metrics[f'road_length_{buffer_dist}m'] = total_road_length
    accessibility_metrics[f'major_roads_{buffer_dist}m'] = major_road_count
    accessibility_metrics[f'road_density_{buffer_dist}m'] = [
        length / (3.14159 * (buffer_dist/1000)**2) if length > 0 else 0 
        for length in total_road_length
    ]

print("✅ Road accessibility metrics calculated for all buffer zones")

=== CALCULATING SPATIAL ACCESSIBILITY METRICS ===

Calculating road accessibility for multiple buffer zones...
   • Processing 100m buffer zone...
   • Processing 250m buffer zone...
   • Processing 250m buffer zone...
   • Processing 500m buffer zone...
   • Processing 500m buffer zone...
✅ Road accessibility metrics calculated for all buffer zones
✅ Road accessibility metrics calculated for all buffer zones


In [7]:
# Calculate property density and market activity metrics
print("=== CALCULATING MARKET & DENSITY METRICS ===")
print()

property_density = []
avg_property_values = []
transaction_counts = []

print("Processing market data for each parcel...")

for idx, parcel in cadastre_projected.iterrows():
    # Find GNAF properties within this parcel
    properties_in_parcel = gnaf_projected[gnaf_projected.within(parcel.geometry)]
    
    # Calculate property density (properties per hectare)
    area_ha = parcel.geometry.area / 10000
    if area_ha > 0:
        density = len(properties_in_parcel) / area_ha
    else:
        density = 0
    property_density.append(density)
    
    # Get transaction data for properties in this parcel
    if len(properties_in_parcel) > 0:
        parcel_gnaf_pids = properties_in_parcel['gnaf_pid'].tolist()
        parcel_transactions = transactions[transactions['gnaf_pid'].isin(parcel_gnaf_pids)]
        
        if len(parcel_transactions) > 0:
            avg_value = parcel_transactions['price'].mean()
            transaction_count = len(parcel_transactions)
        else:
            avg_value = 0
            transaction_count = 0
    else:
        avg_value = 0
        transaction_count = 0
        
    avg_property_values.append(avg_value)
    transaction_counts.append(transaction_count)

# Add market metrics to accessibility analysis
accessibility_metrics['property_density_per_ha'] = property_density
accessibility_metrics['avg_property_value'] = avg_property_values
accessibility_metrics['transaction_count'] = transaction_counts

print("✅ Market and density metrics calculated")
print(f"   • Parcels with transaction data: {sum(1 for x in transaction_counts if x > 0)}")
print(f"   • Average property density: {np.mean(property_density):.1f} properties/hectare")
print(f"   • Average property value: ${np.mean([x for x in avg_property_values if x > 0]):,.0f}")

=== CALCULATING MARKET & DENSITY METRICS ===

Processing market data for each parcel...
✅ Market and density metrics calculated
   • Parcels with transaction data: 335
   • Average property density: 73.8 properties/hectare
   • Average property value: $2,247,635
✅ Market and density metrics calculated
   • Parcels with transaction data: 335
   • Average property density: 73.8 properties/hectare
   • Average property value: $2,247,635


In [8]:
# Create comprehensive investment scoring system
print("=== CREATING COMPREHENSIVE INVESTMENT SCORES ===")
print()

# 1. Calculate Road Accessibility Score (0-100)
score_columns = ['roads_count_250m', 'road_density_250m', 'major_roads_500m', 'road_length_500m']

scaler = MinMaxScaler(feature_range=(0, 100))
accessibility_metrics_scaled = accessibility_metrics[score_columns].copy()
accessibility_metrics_scaled = accessibility_metrics_scaled.fillna(0).replace([np.inf, -np.inf], 0)

scaled_values = scaler.fit_transform(accessibility_metrics_scaled)
scaled_df = pd.DataFrame(scaled_values, columns=[f'{col}_scaled' for col in score_columns])

# Weighted accessibility score
accessibility_weights = {
    'roads_count_250m_scaled': 0.25,    # Nearby road count
    'road_density_250m_scaled': 0.30,   # Road density
    'major_roads_500m_scaled': 0.25,    # Major road access  
    'road_length_500m_scaled': 0.20     # Overall road network
}

accessibility_metrics['accessibility_score'] = sum(
    scaled_df[col] * weight for col, weight in accessibility_weights.items()
)

# 2. Calculate Market Activity Score (0-100)
accessibility_metrics['market_activity_score'] = accessibility_metrics['transaction_count'].clip(upper=10) * 10

# 3. Calculate Property Value Score (0-100)
non_zero_values = accessibility_metrics[accessibility_metrics['avg_property_value'] > 0]['avg_property_value']
if len(non_zero_values) > 0:
    accessibility_metrics['property_value_score'] = accessibility_metrics['avg_property_value'].apply(
        lambda x: 0 if x == 0 else (x / non_zero_values.max()) * 100
    )
else:
    accessibility_metrics['property_value_score'] = 0

# 4. Calculate Development Density Score (0-100)  
max_density = accessibility_metrics['property_density_per_ha'].max()
if max_density > 0:
    accessibility_metrics['density_score'] = (accessibility_metrics['property_density_per_ha'] / max_density * 100)
else:
    accessibility_metrics['density_score'] = 0

print("✅ Individual component scores calculated:")
print(f"   • Accessibility scores: 0-{accessibility_metrics['accessibility_score'].max():.1f}")
print(f"   • Market activity scores: 0-{accessibility_metrics['market_activity_score'].max():.1f}")
print(f"   • Property value scores: 0-{accessibility_metrics['property_value_score'].max():.1f}")
print(f"   • Density scores: 0-{accessibility_metrics['density_score'].max():.1f}")

=== CREATING COMPREHENSIVE INVESTMENT SCORES ===

✅ Individual component scores calculated:
   • Accessibility scores: 0-85.3
   • Market activity scores: 0-100.0
   • Property value scores: 0-100.0
   • Density scores: 0-100.0


In [9]:
# Calculate the final comprehensive investment score
print("=== COMPREHENSIVE INVESTMENT SCORE CALCULATION ===")
print()

# Create comprehensive score combining all factors
comprehensive_weights = {
    'accessibility_score': 0.30,      # Road connectivity (30%)
    'market_activity_score': 0.25,   # Transaction volume (25%) 
    'property_value_score': 0.20,    # Property values (20%)
    'density_score': 0.25            # Development density (25%)
}

accessibility_metrics['comprehensive_score'] = sum(
    accessibility_metrics[metric] * weight 
    for metric, weight in comprehensive_weights.items()
)

# Create investment grade categories
def get_comprehensive_grade(score):
    if score >= 80: return 'AAA+ Elite Investment'
    elif score >= 70: return 'AA+ Premium Investment' 
    elif score >= 60: return 'A+ Excellent Investment'
    elif score >= 50: return 'A Good Investment'
    elif score >= 40: return 'B+ Above Average'
    elif score >= 30: return 'B Average Potential'
    else: return 'C Consider Carefully'

accessibility_metrics['comprehensive_grade'] = accessibility_metrics['comprehensive_score'].apply(get_comprehensive_grade)

# Add simple accessibility grades too
def get_accessibility_grade(score):
    if score >= 80: return 'A+ Excellent'
    elif score >= 70: return 'A Good' 
    elif score >= 60: return 'B+ Above Average'
    elif score >= 50: return 'B Average'
    elif score >= 40: return 'C+ Below Average'
    elif score >= 30: return 'C Poor'
    else: return 'D Very Poor'

accessibility_metrics['accessibility_grade'] = accessibility_metrics['accessibility_score'].apply(get_accessibility_grade)

print("✅ Comprehensive scoring complete!")
print()
print("INVESTMENT GRADE DISTRIBUTION:")
print(accessibility_metrics['comprehensive_grade'].value_counts().sort_index())

=== COMPREHENSIVE INVESTMENT SCORE CALCULATION ===

✅ Comprehensive scoring complete!

INVESTMENT GRADE DISTRIBUTION:
comprehensive_grade
A Good Investment          8
B Average Potential       16
B+ Above Average           8
C Consider Carefully    1262
Name: count, dtype: int64


In [10]:
# Generate investment analysis results and insights
print("=== INVESTMENT ANALYSIS RESULTS ===")
print()

# Show top investment opportunities  
print("🏆 TOP 15 INVESTMENT OPPORTUNITIES:")
top_comprehensive = accessibility_metrics.nlargest(15, 'comprehensive_score')[[
    'sa4', 'area_hectares', 'comprehensive_score', 'comprehensive_grade',
    'accessibility_score', 'property_density_per_ha', 'avg_property_value', 
    'transaction_count', 'roads_count_250m', 'major_roads_500m'
]].round(2)
print(top_comprehensive.to_string(index=False))
print()

# Regional analysis
print("📍 REGIONAL PERFORMANCE SUMMARY:")
regional_summary = accessibility_metrics.groupby('sa4').agg({
    'comprehensive_score': 'mean',
    'accessibility_score': 'mean', 
    'market_activity_score': 'mean',
    'property_value_score': 'mean',
    'area_hectares': 'mean'
}).round(1)
print(regional_summary.to_string())
print()

# Investment grade analysis
print("💎 INVESTMENT GRADE DISTRIBUTION:")
grade_analysis = accessibility_metrics.groupby('comprehensive_grade').agg({
    'comprehensive_score': ['mean', 'count'],
    'avg_property_value': 'mean',
    'accessibility_score': 'mean'
}).round(1)
grade_analysis.columns = ['avg_score', 'count', 'avg_value', 'avg_accessibility']
print(grade_analysis.to_string())
print()

# Property size vs performance
print("📏 PROPERTY SIZE vs INVESTMENT PERFORMANCE:")
size_bins = [0, 0.05, 0.1, 0.5, 1, 50]
size_labels = ['Micro (<0.05ha)', 'Small (0.05-0.1ha)', 'Medium (0.1-0.5ha)', 'Large (0.5-1ha)', 'Very Large (1ha+)']
accessibility_metrics['size_category'] = pd.cut(accessibility_metrics['area_hectares'], bins=size_bins, labels=size_labels)

size_analysis = accessibility_metrics.groupby('size_category').agg({
    'comprehensive_score': 'mean',
    'accessibility_score': 'mean',
    'avg_property_value': 'mean'
}).round(1)
print(size_analysis.to_string())

=== INVESTMENT ANALYSIS RESULTS ===

🏆 TOP 15 INVESTMENT OPPORTUNITIES:
                              sa4  area_hectares  comprehensive_score comprehensive_grade  accessibility_score  property_density_per_ha  avg_property_value  transaction_count  roads_count_250m  major_roads_500m
Sydney - North Sydney and Hornsby           0.11                54.25   A Good Investment                57.48                   332.16          1489230.77                 13                41                 6
Sydney - North Sydney and Hornsby           0.11                54.25   A Good Investment                57.48                   332.16          1489230.77                 13                41                 6
Sydney - North Sydney and Hornsby           0.11                54.25   A Good Investment                57.48                   332.16          1489230.77                 13                41                 6
Sydney - North Sydney and Hornsby           0.11                54.25   A Good Inves

C:\Users\danie\AppData\Local\Temp\ipykernel_8188\3668632606.py:44: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  size_analysis = accessibility_metrics.groupby('size_category').agg({


In [11]:
# Final summary and export results
print("🏠 PROPERTY INVESTMENT ANALYSIS COMPLETE")
print("="*60)
print()

# Create final export dataframe
final_results = accessibility_metrics[[
    'state', 'sa4', 'area_hectares',
    'comprehensive_score', 'comprehensive_grade',
    'accessibility_score', 'accessibility_grade',
    'market_activity_score', 'property_value_score', 'density_score',
    'property_density_per_ha', 'avg_property_value', 'transaction_count',
    'roads_count_100m', 'roads_count_250m', 'roads_count_500m',
    'major_roads_500m', 'road_density_250m'
]].copy()

# Summary statistics
total_parcels = len(final_results)
investment_grade = len(final_results[final_results['comprehensive_score'] >= 60])
premium_grade = len(final_results[final_results['comprehensive_score'] >= 70])

print("📊 ANALYSIS SUMMARY:")
print(f"   • Total property parcels analyzed: {total_parcels:,}")
print(f"   • Road network segments: {len(roads):,}")  
print(f"   • Property addresses (GNAF): {len(gnaf_prop):,}")
print(f"   • Transaction records: {len(transactions):,}")
print(f"   • Investment grade properties (60+): {investment_grade:,} ({investment_grade/total_parcels*100:.1f}%)")
print(f"   • Premium opportunities (70+): {premium_grade:,} ({premium_grade/total_parcels*100:.1f}%)")
print(f"   • Average comprehensive score: {final_results['comprehensive_score'].mean():.1f}")
print()

print("🎯 INVESTMENT METRICS CREATED:")
print("   1. Road Accessibility Score (0-100) - Transport connectivity")
print("   2. Market Activity Score (0-100) - Transaction volume")  
print("   3. Property Value Score (0-100) - Relative market values")
print("   4. Development Density Score (0-100) - Growth potential")
print("   5. Comprehensive Investment Score (0-100) - Combined rating")
print()

print("💡 KEY INSIGHTS FOR AUSTRALIAN PROPERTY INVESTORS:")
print("   • Focus on comprehensive scores 60+ for investment potential")
print("   • Sydney North/Hornsby region shows strongest performance")
print("   • Transport accessibility crucial for rental demand & values")
print("   • Market activity indicates investor confidence levels")
print("   • Property density signals development & growth opportunities")
print()

print("✅ READY FOR INVESTMENT DECISIONS!")
print("Use the final_results dataframe to compare properties and make data-driven investment choices.")

# Display data shape for confirmation
print(f"\nFinal dataset shape: {final_results.shape}")
print("Available columns:", list(final_results.columns))

🏠 PROPERTY INVESTMENT ANALYSIS COMPLETE

📊 ANALYSIS SUMMARY:
   • Total property parcels analyzed: 1,294
   • Road network segments: 173
   • Property addresses (GNAF): 70,591
   • Transaction records: 5,576
   • Investment grade properties (60+): 0 (0.0%)
   • Premium opportunities (70+): 0 (0.0%)
   • Average comprehensive score: 13.7

🎯 INVESTMENT METRICS CREATED:
   1. Road Accessibility Score (0-100) - Transport connectivity
   2. Market Activity Score (0-100) - Transaction volume
   3. Property Value Score (0-100) - Relative market values
   4. Development Density Score (0-100) - Growth potential
   5. Comprehensive Investment Score (0-100) - Combined rating

💡 KEY INSIGHTS FOR AUSTRALIAN PROPERTY INVESTORS:
   • Focus on comprehensive scores 60+ for investment potential
   • Sydney North/Hornsby region shows strongest performance
   • Transport accessibility crucial for rental demand & values
   • Market activity indicates investor confidence levels
   • Property density signals

In [15]:
final_results.to_excel('Property_Investment_Analysis_Results.xlsx', index=False)

In [13]:
# Export final results to pickle file for easy reuse
import pickle
import os

# Create output directory if it doesn't exist
output_dir = './'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Export the complete final results dataframe
pkl_file_path = os.path.join(output_dir, 'Dataset.pkl')
final_results.to_pickle(pkl_file_path)
